# Flight Data Data Cleaning (Legendary)

NOTE: This documenation is old, and the data export associated with it is old as well. It exists for the sake of producing old information for past versions of my index.html.


## Data Import and Inspection

This is the documentation for the 2006-2010 Flight Dataset. In this document, we generate a new feature, impute ill inputed/converted data values, and finalize data properties and features for data analysis implementation.

In [1]:
from zipfile import ZipFile as zp
import numpy as np
import pandas as pd
import string

import time

import matplotlib.pyplot as plt

## Import

In [2]:
def get_csv_file(filename):
    t0 = time.time()
    with zp('../Data/RawData/{}.zip'.format(filename)) as flight_zpfl:
        with flight_zpfl.open('{}.csv'.format(filename)) as f_info:
            dataframe =pd.read_csv(f_info,delimiter=',')
    print("Import loading time was {} seconds".format(time.time()-t0))
    return(dataframe)

In [3]:
df_flights = get_csv_file('783548897_52017_1328_airline_delay_causes')

Import loading time was 1.172903299331665 seconds


### Data Inspection 

This section observes several attributes of the flight dataset.

In [4]:
df_flights.shape

(91837, 22)


This dataset contains 91837 rows and 22 features.

In [5]:
df_flights.head(2)

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,Unnamed: 21
0,2006,8,AA,American Airlines Inc.,ABQ,"Albuquerque, NM: Albuquerque International Sun...",310.0,73.0,17.53,8.83,...,26.51,5.0,1.0,3742.0,838.0,585.0,729.0,21.0,1569.0,NaN
1,2006,8,AA,American Airlines Inc.,ANC,"Anchorage, AK: Ted Stevens Anchorage Internati...",62.0,38.0,11.53,0.88,...,11.27,0.0,0.0,2605.0,879.0,100.0,870.0,0.0,756.0,NaN


We observe a random space " " in two feature titles (e.g. " month"). The following checks if some space exists in a feature title, and removes it.

In [6]:
col_names = list(df_flights.columns)
col_names_new =[]
for i in col_names:
    col_names_new.append(i.strip(" "))

In [7]:
print(col_names_new)

['year', 'month', 'carrier', 'carrier_name', 'airport', 'airport_name', 'arr_flights', 'arr_del15', 'carrier_ct', 'weather_ct', 'nas_ct', 'security_ct', 'late_aircraft_ct', 'arr_cancelled', 'arr_diverted', 'arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay', 'security_delay', 'late_aircraft_delay', 'Unnamed: 21']


In [8]:
df_flights.columns = col_names_new

The above list confirms corrections of existing whitespace in feature titles. We now replace this list of corrected titles to the old flight dataframe's title.

In [9]:
df_flights.dtypes

year                     int64
month                    int64
carrier                 object
carrier_name            object
airport                 object
airport_name            object
arr_flights            float64
arr_del15              float64
carrier_ct             float64
weather_ct             float64
nas_ct                 float64
security_ct            float64
late_aircraft_ct       float64
arr_cancelled          float64
arr_diverted           float64
arr_delay              float64
carrier_delay          float64
weather_delay          float64
nas_delay              float64
security_delay         float64
late_aircraft_delay    float64
Unnamed: 21            float64
dtype: object

Lastly, we provide some descriptive statistics for the "df_flights" dataset.

In [10]:
df_flights.describe()

,year,month,arr_flights,arr_del15,carrier_ct,weather_ct,nas_ct,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,Unnamed: 21
count,91837.000000,91837.000000,91653.000000,91623.000000,91653.000000,91653.000000,91653.000000,91653.000000,91653.000000,91653.000000,91653.000000,91653.000000,91653.000000,91653.000000,91653.000000,91653.000000,91653.000000,0.0
mean,2007.968847,6.492198,376.501675,80.083298,22.397971,2.980848,27.622592,0.212576,26.843168,6.795271,0.889987,4408.519972,1253.141185,232.490448,1270.455904,7.625162,1644.807273,NaN
std,1.399134,3.445474,1008.399764,221.292022,49.784500,11.647614,98.159085,1.026079,82.746186,27.862032,4.245160,13404.644997,3327.285102,954.124065,5478.810428,41.204998,5192.477674,NaN
min,2006.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,NaN
25%,2007.000000,4.000000,62.000000,12.000000,4.320000,0.000000,2.290000,0.000000,1.960000,0.000000,0.000000,572.000000,202.000000,0.000000,79.000000,0.000000,94.000000,NaN
50%,2008.000000,6.000000,125.000000,28.000000,10.000000,0.810000,6.870000,0.000000,6.790000,1.000000,0.000000,1401.000000,508.000000,35.000000,252.000000,0.000000,399.000000,NaN
75%,2009.000000,9.000000,269.000000,62.000000,21.710000,2.390000,17.510000,0.000000,18.670000,5.000000,1.000000,3287.000000,1137.000000,180.000000,685.000000,0.000000,1181.000000,NaN
max,2010.000000,12.000000,15993.000000,4966.000000,1792.070000,641.540000,2739.180000,80.560000,1885.470000,1283.000000,248.000000,356883.000000,134693.000000,57707.000000,130920.000000,3119.000000,145680.000000,NaN


## Data Imputation

### Checking for invalid entries
We verify no invalid entries our within our dataset. Such entries we may resolve are "NaN," corrupt, or "NA" values.

In [11]:
df_flights.isnull().sum()

year                       0
month                      0
carrier                    0
carrier_name               0
airport                    0
airport_name               0
arr_flights              184
arr_del15                214
carrier_ct               184
weather_ct               184
nas_ct                   184
security_ct              184
late_aircraft_ct         184
arr_cancelled            184
arr_diverted             184
arr_delay                184
carrier_delay            184
weather_delay            184
nas_delay                184
security_delay           184
late_aircraft_delay      184
Unnamed: 21            91837
dtype: int64

In [12]:
df_flights[df_flights['arr_flights'].isnull()== True]

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,Unnamed: 21
414,2006,1,EV,Atlantic Southeast Airlines,MCI,"Kansas City, MO: Kansas City International",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1137,2006,1,TZ,ATA Airlines d/b/a ATA,BWI,"Baltimore, MD: Baltimore/Washington Internatio...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2337,2006,2,OH,Comair Inc.,GNV,"Gainesville, FL: Gainesville Regional",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2347,2006,2,OH,Comair Inc.,ICT,"Wichita, KS: Wichita Dwight D Eisenhower National",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2397,2006,2,OH,Comair Inc.,TRI,"Bristol/Johnson City/Kingsport, TN: Tri-Cities...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3425,2006,3,EV,Atlantic Southeast Airlines,ONT,"Ontario, CA: Ontario International",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5619,2006,4,RU,ExpressJet Airlines Inc.,SBN,"South Bend, IN: South Bend International",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6797,2006,5,OH,Comair Inc.,AVL,"Asheville, NC: Asheville Regional",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6834,2006,5,OH,Comair Inc.,FWA,"Fort Wayne, IN: Fort Wayne International",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6844,2006,5,OH,Comair Inc.,ICT,"Wichita, KS: Wichita Dwight D Eisenhower National",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


We observe that from the arr_flights column to the late_aircraft_delay column, there are ~146 ill entries, per column. Moreover, we observe 59112 bad entries in Unnamed:21.

Because of the $\dfrac{184}{\text{Dataset Size}}=\dfrac{184}{91837}=.200\%$ missing entries per column, we can proceed with removing them, without affecting future analyses.

Even more, we have to removed the entire "Unnamed:21" column. With the lack of existence of entries in this column, we can simply disregard/delete it.


The following will remove the ill-entred values:

In [13]:
df_flights = df_flights[df_flights['arr_flights'].isnull()== False]


In [14]:
df_flights_col_noUnamed21 = list(df_flights.columns)
df_flights_col_noUnamed21.pop(-1)

'Unnamed: 21'

In [15]:
df_flights=df_flights[df_flights_col_noUnamed21]

In [16]:
df_flights.shape

(91653, 21)

In [17]:
df_flights = df_flights[df_flights['arr_del15'].isnull()== False]

In [18]:
df_flights.shape

(91623, 21)


The above is a confirmation of removing the ill data-filled "Unnamed:21" feature.

In [19]:
df_flights.isnull().sum()

year                   0
month                  0
carrier                0
carrier_name           0
airport                0
airport_name           0
arr_flights            0
arr_del15              0
carrier_ct             0
weather_ct             0
nas_ct                 0
security_ct            0
late_aircraft_ct       0
arr_cancelled          0
arr_diverted           0
arr_delay              0
carrier_delay          0
weather_delay          0
nas_delay              0
security_delay         0
late_aircraft_delay    0
dtype: int64

In [20]:
df_flights.head(2)


,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_del15,carrier_ct,weather_ct,...,security_ct,late_aircraft_ct,arr_cancelled,arr_diverted,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay
0,2006,8,AA,American Airlines Inc.,ABQ,"Albuquerque, NM: Albuquerque International Sun...",310.0,73.0,17.53,8.83,...,1.0,26.51,5.0,1.0,3742.0,838.0,585.0,729.0,21.0,1569.0
1,2006,8,AA,American Airlines Inc.,ANC,"Anchorage, AK: Ted Stevens Anchorage Internati...",62.0,38.0,11.53,0.88,...,0.0,11.27,0.0,0.0,2605.0,879.0,100.0,870.0,0.0,756.0


We can now confirm the following:

1. Corrected feature titles

2. All data entries have been imputed

3. We have removed the bad feature "Unnamed:21"

### Selecting Features

In [21]:
df_flights.dtypes

year                     int64
month                    int64
carrier                 object
carrier_name            object
airport                 object
airport_name            object
arr_flights            float64
arr_del15              float64
carrier_ct             float64
weather_ct             float64
nas_ct                 float64
security_ct            float64
late_aircraft_ct       float64
arr_cancelled          float64
arr_diverted           float64
arr_delay              float64
carrier_delay          float64
weather_delay          float64
nas_delay              float64
security_delay         float64
late_aircraft_delay    float64
dtype: object

In [22]:
semi_final_feature = [ 'year', 'month', 'carrier', 'carrier_name',
                 'airport', 'airport_name','arr_flights','arr_cancelled', 
                 'arr_delay', 'carrier_delay', 'weather_delay', 
                 'nas_delay', 'security_delay',
                 'late_aircraft_delay']
df_flights_semi_final = df_flights[semi_final_feature ]

We observe the data types for a majority of the features are correct. However, in hindsight, our Month and Year features are not quite dates. I.e., Two features in our dataset are not of "Date" types.

The following creates a new "Date" feature in our dataset, for time series analysis.

### Add Date Feature

In [23]:
df_flights_semi_final['Date'] = pd.to_datetime(
    dict(year = df_flights_semi_final['year'],
         month = df_flights_semi_final['month'],day =1)
                                    )

C:\Anaconda3\lib\site-packages\ipykernel\__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [24]:

df_flights_semi_final.head(2)

,year,month,carrier,carrier_name,airport,airport_name,arr_flights,arr_cancelled,arr_delay,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,Date
0,2006,8,AA,American Airlines Inc.,ABQ,"Albuquerque, NM: Albuquerque International Sun...",310.0,5.0,3742.0,838.0,585.0,729.0,21.0,1569.0,2006-08-01
1,2006,8,AA,American Airlines Inc.,ANC,"Anchorage, AK: Ted Stevens Anchorage Internati...",62.0,0.0,2605.0,879.0,100.0,870.0,0.0,756.0,2006-08-01


### Add Proportion Representation of Selected Features

Learning from Patrick Senti's [Flight Data project](https://miraculixx.github.io/flightdelays/index.html) and feedback to us, we create several features that are proportional represenations of the delay features we established above.

I.e., We convert several flight delay features into proportions by month or by year.

The following is a function that takes in an array of selected features with its respected dataset & time type, and outputs a new dataframe with the new features by grouped proportional times. .

In [57]:
'''
converts a given set of columns into proportions by some time (year or month) for the flight dataset above
'''
def to_proportion(data, selected_features,time_type):
    #Make a copy of original dataframe
    dataset_new = pd.DataFrame(data)

    #Partition data for delay by minutes metrics and delay count metrics
    selected_features_mins = selected_features[:] ##6 features
    selected_features_ct = selected_features[:15] ##15 features
    
    
    '''
    Generated titles for new features, by proportion
    
    I.e. Names + proportion_by_timeType== new features
    '''
    
    selected_feat_names = []
    for i in range(0,len(selected_features)):
        selected_feat_names.append("{name}_prop_by_{time}".format(name = selected_features[i], time = time_type))
    
    '''
    Receive a list of the time values in our time_type
    
    E.x. Years is an array 2006,2007,2008,2009,2010
    '''
    time_list = list(dataset_new[time_type].unique())
    
    
    
    #Empty dictionary to store our arrays with "selected_feat_names" keys
    dictionary = {}
    for i in selected_feat_names:
        dictionary[i] = []
        
    '''
    This remaining portion considers the selected features, list of type of times, 
    and converts features to proportions with respect to some time(year or month)
    '''    
    '''
    get features....loop features....group datasum...loop data'''
    
    #Loop through our selected features
    for i, val in enumerate(selected_features):
        
        #Loop through the unique time entries
        for i_time, i_val in enumerate(time_list):
            
            #Get sum of values for some time i_val in time_list and considered column val
            temp_ds_sum = dataset_new.groupby(time_type).sum()[val][i_val]
            
            #loop through dataset
            for i_ds, row_ds in dataset_new.iterrows():
                
                #If the row's time is equal to time we are considering, then convert to proportion
                #with respect to time i_val
                if row_ds[time_type] == i_val:
                    dictionary[selected_feat_names[i]].append(row_ds[val]/temp_ds_sum)
        
        
        #Add lists from dictionary to dataframe            
        dataset_new[selected_feat_names[i]] = dictionary[selected_feat_names[i]]
    
    return dataset_new

In [58]:
df_flights.columns[6:21]

Index(['arr_flights', 'arr_del15', 'carrier_ct', 'weather_ct', 'nas_ct',
       'security_ct', 'late_aircraft_ct', 'arr_cancelled', 'arr_diverted',
       'arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay',
       'security_delay', 'late_aircraft_delay'],
      dtype='object')

In [62]:

last_feat = [ 'arr_flights', 'arr_cancelled',
       'arr_delay', 'carrier_delay', 'weather_delay', 'nas_delay',
       'security_delay', 'late_aircraft_delay']
df_flights_semi_final_with_prop = to_proportion(df_flights_semi_final, last_feat, "year")

KeyboardInterrupt: 

In [ ]:
df_flights_semi_final_with_both_prop = to_proportion(df_flights_semi_final_with_prop, last_feat, "month")


In [ ]:
df_flights_semi_final_with_both_prop.head(1)


## Export Data

We partition the "df_flights_semi_final_with_prop" dataset into the following:

1. df_flights_final: The flight data in minutes

2. df_flights_final_proportion_byYear: The flight data in proportion of minutes to total delay minutes in its respective year

3. df_flights_final_proportion_byMonth: The flight data in proportion of minutes to total delay minutes in its respective month

### Flight Data (in minutes)

In [ ]:
final_feat = [ 'year','month','carrier','carrier_name','airport','airport_name','arr_flights','arr_cancelled', 
                 'arr_delay', 'carrier_delay', 'weather_delay', 
                 'nas_delay', 'security_delay',
                 'late_aircraft_delay','Date']
df_flights_final = df_flights_semi_final_with_both_prop[final_feat]

### Flight Data (Proportion by Years)

In [ ]:
final_featuresProportion_byYear = [ 'year','month','carrier','carrier_name','airport','airport_name',
                                    'arr_flights_prop_by_year','arr_cancelled_prop_by_year', 'arr_delay_prop_by_year', 
                                   'carrier_delay_prop_by_year', 'weather_delay_prop_by_year', 'nas_delay_prop_by_year', 
                                   'security_delay_prop_by_year','late_aircraft_delay_prop_by_year','Date']
df_flights_final_proportion_byYear = df_flights_semi_final_with_both_prop[final_featuresProportion_byYear]

### Flight Data (proportion by Months)

In [ ]:
final_featuresProportion_byMonth = ['year','month','carrier','carrier_name','airport','airport_name', 
                                    'arr_flights_prop_by_month','arr_cancelled_prop_by_month', 'arr_delay_prop_by_month', 
                                    'carrier_delay_prop_by_month', 'weather_delay_prop_by_month', 'nas_delay_prop_by_month', 
                                    'security_delay_prop_by_month','late_aircraft_delay_prop_by_month']
df_flights_final_proportion_byMonth = df_flights_semi_final_with_both_prop[final_featuresProportion_byMonth]

### Export CSV Data

In [ ]:
df_flights_final.to_csv('../Data/PreparedData/flight_data_OLD.csv',sep=',', header=True)

df_flights_final_proportion_byYear.to_csv('../Data/PreparedData/flight_data_byYear_OLD.csv',sep=',', header=True)

df_flights_final_proportion_byMonth.to_csv('../Data/PreparedData/flight_data_byMonth_OLD.csv',sep=',', header=True)

## Data Dictionary

1. Year: 2006-2010

2. Month: 1-12

3. DayofMonth: 1-31

4. DayOfWeek: 1 (Monday) - 7 (Sunday)

5. DepTime: actual departure time (local, hhmm)

6. CRSDepTime: scheduled departure time (local, hhmm)

7. ArrTime: actual arrival time (local, hhmm)

8. CRSArrTime: scheduled arrival time (local, hhmm)

9. UniqueCarrier: unique carrier code

10. FlightNum: flight number

11. TailNum: plane tail number

12. ActualElapsedTime: in minutes

13. CRSElapsedTime: in minutes

14. AirTime: in minutes

15. ArrDelay: arrival delay, in minutes

16. DepDelay: departure delay, in minutes

17. Origin: origin IATA airport code

18. Dest: destination IATA airport code

19. Distance in miles

20. axiIn: taxi in time, in minutes

21. TaxiOut: taxi out time in minutes

22. Cancelled: was the flight cancelled?

23. CancellationCode:reason for cancellation (A = carrier, B = weather, C = NAS, D = security)

24. Diverted: 1 = yes, 0 = no

25. CarrierDelay: in minutes

26. WeatherDelay: in minutes

27. NASDelay: in minutes

28. SecurityDelay: in minutes

29. LateAircraftDelay: in minutes
